In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage as ndi
from skimage.feature import shape_index, match_template
from skimage.draw import circle
import skimage.io as io
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, black_tophat, skeletonize, convex_hull_image, disk
from scipy.ndimage.morphology import distance_transform_bf
from skimage.exposure import rescale_intensity

%matplotlib inline

#This function compares two images for visualization purposes
#Params
    # original, image 2d ndarray
    # filtered, image 2d ndarray
    # filter_name, string
def plot_comparion(original, filtered, filter_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2,figsize=(16,16),sharex=True,sharey=True)
    
    ax1.imshow(original)
    ax1.set_title('Original')
    ax1.axis('off')
    ax2.imshow(filtered)
    ax2.set_title(filter_name)
    ax2.axis('off')

In [1]:
#This function accepts and image and calculates the degree of punctaness in the image
#Params 
    #filename, string path to image file
    #radii, int, determines the disk size for the wth algorithm. See scikit-image morphological filtering tutorial
    # thresh, int, sets global threshold for background and foreground objects. Can be imporoved by making local thresholding
    #Output, int the geometric sum of all pixel intensities considered puncta
def measure_punctaness(filename,radii,thresh): #Threshold should be a low number, less than 10
    image = io.imread(filename)
    image = rescale_intensity(image,in_range='image')
    cell = disk(radii)
    opened = opening(image,cell) #erosion followed by dilation
    wth = image - opened #definition of tophat function
    mask = wth<thresh
    wth[mask] = 0
    sum2 = np.sum(wth)
    return sum2

#This Block will name all the bf channels
fbaselist = ['C4_LAMMDAS_dopamine_WTXY','C5_LAMMDAS_dopamine_A53TXY','C6_LAMMDAS_dopamine_A30PXY','C7_LAMMDAS_dopamine_E46KXY',\
            'C8_LAMMDAS_dopamine_gfpXY','E4_LAMMDAS_apo_WTXY','E5_LAMMDAS_apo_A53TXY','E6_LAMMDAS_apo_A30PXY','E7_LAMMDAS_apo_E46KXY',\
            'E8_LAMMDAS_apo_gfpXY']

#Parameters Used disk raddi = 16, threshold = 10
#This script calculates the punctaness for each input image and writes it to a an ouput file.
#It calculates the punctaness for each image in a 5 image stack for a given condition
def write_puncta():
    fpath = '/Users/ajipalar/Documents/UCSF/Classes/PUBS/LAMMDAS/Microscopy/20181002_160910_824/nd2as8bit/'
    output_fname = '/Users/ajipalar/Documents/UCSF/Classes/PUBS/LAMMDAS/Microscopy/20181002_160910_824/OutputData/punctaness.txt'
    output_file = open(output_fname, 'w')
    for fbase in fbaselist[0:3]:
        for stack in range(1,6):
            fname = fbase+str(stack)+'C1.jp2'
            fullpath = fpath+fname
            punctaness = measure_punctaness(fullpath,16,10)
            output_file.write(fname + ', ' + str(punctaness))